In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/subfolder/Global_local

In [ ]:
import os
import numpy as np
import pickle
import torch
from torch.optim import Adam
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import DataLoader
import time
import h5py
import imutils
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torchvision
from torch.optim import lr_scheduler
# from misc_functions import preprocess_image, recreate_image, save_image

In [ ]:
# read pickle file
with open('Customized_Gratings/Small_Gratings.pkl','rb') as f:
  gratings = pickle.load(f)

In [ ]:
cls = 9
times = 10
#train index
num = len(gratings)
div = 180/cls
index_tmp = np.repeat(np.arange(num),cls*cls)
k = []
for i in range(cls):
  k.extend([i*div])
k = k*num*cls
l = []
for i in range(cls):
  l.extend([i*div])
l = l*num
l = np.repeat(l,cls)
data_index = [i for i in zip(index_tmp,k,l)] 

In [ ]:
## grating based on requirements specified in index

class myDataset():
    """
    Args:
              input input_file_list = [f_content, f_angle]
              target_file_list = f_trans      
              patch_size: int , the cubic patch parameter     
    """
    def __init__(self,input_patches,index,cls = cls, input_size=224,random = False):

        self.index = index
        self.inputs = input_patches
        self.input_size = input_size
        self.div = 180/cls
       
    def __len__(self):
        return len(self.index)

    @staticmethod
    def rotate(img,angle):
        return imutils.rotate(img, angle=angle)

    @staticmethod
    ##Randomly distributed with minimum distance
    def random_distributed_with_minimum(n_points,half_distance,boundary_length):
      position = []
      occupied_length = 0
      remain_length = boundary_length 
      def set_first_remain_point(remain_n,remain_length):
          return random.randint(half_distance, remain_length - remain_n * half_distance *2 - half_distance) + occupied_length
      for i in range(n_points,0,-1):
          remain_n = i - 1
          p = set_first_remain_point(remain_n,remain_length)
          occupied_length = p + half_distance
          remain_length = boundary_length - occupied_length
          position.append(p)
      sign = random.randint(0,1)
      if sign:
        position =[(boundary_length - i) for i in position]
      return position
    def __getitem__(self, item):
        global p,img_0,n,img0,a
        idx, angle_l, angle_g = self.index[item]  
        if item == 0 or self.index[item-1][0] != self.index[item][0]:
            img_0 = self.inputs[idx]
            #uniform with random shading
            img_0 = img_0/255*random.uniform(0.5,1)
            a = img_0.shape[1]  #half of the patch's side length
            #num of gratings & generate random distributed positions
            n = 2 if self.input_size//a == 2 else random.randint(2,self.input_size//a)
            p = self.random_distributed_with_minimum(n,np.int(a//2),self.input_size)
            #Initialize with Gaussian noise
            noise_sigma = random.uniform(0,0.3)
            img0 = np.random.randn(3,self.input_size,self.input_size) * noise_sigma
        
        #random rotate locally 
        img = self.rotate(img_0.swapaxes(0,2),angle_l-angle_g).swapaxes(0,2)

        img_1 = np.zeros((3,self.input_size,self.input_size))
        img_out = img0.copy()
        #merge based on random distributed positions
        for i in p:
          # print(int(a//2),i,'\n')
          img_1[:,int(self.input_size//2-a//2):int(self.input_size//2+a//2),i-int(a//2):i+int(a//2)] += img

        #rotate globally based on label difference
        img_1 = self.rotate(img_1.swapaxes(0,2),angle_g).swapaxes(0,2)
        img_out += img_1
        img_out =  (img_out-np.min(img_out))/(np.max(img_out)-np.min(img_out))
        label_l = np.int(angle_l//self.div)
        label_g = np.int(angle_g//self.div)

        return torch.from_numpy(img_out).float(), label_l,label_g

In [ ]:
batch_size = 275
dataset = myDataset(gratings,data_index)

In [ ]:
# im = []
# plt.figure(figsize = (10,100))
# for i in range(18):
#   # if dataset[i][2] ==0:
#     tmp = dataset[i+600][0].numpy().swapaxes(0,2).swapaxes(0,1)
#     # tmp = imutils.rotate(tmp, angle=90)
#     im.append(tmp)
#     # plt.imshow(imutils.rotate(tmp, angle=90))
#     # plt.show()
# im = np.concatenate(im,axis = 1)
# plt.imshow(im)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
## 17size
import torch.nn as nn
import torch
from torch import autograd

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch), 
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, input):
        return self.conv(input)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv0 = nn.Conv2d(3, 16, 7, stride=2, padding = 4)  #224
        self.pool0 = nn.MaxPool2d(3,2)
        self.conv1 = DoubleConv(16, 32)  # 56
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(32, 64) # 28
        self.pool2 = nn.MaxPool2d(2)
        # self.conv3 = DoubleConv(64, 128) # 14
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.pool3 = nn.MaxPool2d(2)
        # self.conv4 = DoubleConv(128, 256) # 7
        self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)

        # self.pool4 = nn.MaxPool3d(2)
        # self.conv5 = DoubleConv(256, 512)

        self.fc1 = nn.Linear(256*7*7, 256)
        self.fc2 = nn.Linear(256, cls)
        self.dropout = nn.Dropout(0.5)
        self.dropout0 = nn.Dropout(0.25)
        # self.dropout1 = nn.Dropout(0.5)
    def forward(self, x):
        c0 = self.conv0(x)     
        p0 = F.relu(self.pool0(c0))
        # print(p0.shape)
        c1 = self.conv1(p0)
        p1 = self.pool1(c1)
        # p1 = self.dropout0(p1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        p2 = self.dropout(p2)
        c3 = self.conv3(p2)
        p3 = F.relu(self.pool3(c3))
        p3 = self.dropout(p3)
        c4 = F.relu(self.conv4(p3))
        out = c4.view(-1, 256*7*7)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        # out = nn.LogSoftmax(dim = 1)(x)
        return out

model = CNN()
model.cuda()
model.train()

from torchsummary import summary
summary(model, input_size=(3,224,224))
criterion = nn.CrossEntropyLoss()


In [ ]:
# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
id = 851

In [ ]:
data, l, g = dataset[id]
# plt.imshow(data.numpy().swapaxes(0,2))
data.unsqueeze_(0)
data = data.cuda()
plt.imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))

In [ ]:
sort = True 

Multi-angle


In [ ]:
img = []
idx = 810+9
fig = plt.figure(figsize = (5*cls,5))
ax = fig.subplots(1,cls)
for i in range(cls):
  id = i + idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  img.append(data)
  ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i].axis('off')

In [ ]:
img = img1

In [ ]:
im = []

t = 1 # No. x maximum
fig = plt.figure(figsize = (5,cls))
ax = fig.subplots(cls,5)
plt.suptitle('Local_limited',y = 1)

cols = ['Input(Stimuli)','Conv1','Conv2','Conv3','Conv4']
for axes, col in zip(ax[0], cols):
    axes.set_title(col)
for i in range(9):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  # model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  # model.eval()
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])

for i in range(cls):
  for j in range(4):
      ax[i,j+1].imshow(im[i*4+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 

In [ ]:
im = []
t = 1 # No. x maximum
k = 50 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
plt.suptitle('Global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

In [ ]:
img = []
img1 = []
idx = 810
fig = plt.figure(figsize = (5*cls,5))
ax = fig.subplots(1,cls)
for i in range(cls):
  id = i*9 + idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  img.append(data)
  if i == 2:
    img1.append(data)
    for j in range(1,cls):
      id = i*9 + idx + j
      data, l, g = dataset[id]
      data.unsqueeze_(0)
      data = data.cuda()
      img1.append(data)
  
  ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i].axis('off')

In [ ]:
fig = plt.figure(figsize = (5*cls,5))
ax = fig.subplots(1,cls)  
for i in range(cls):
  ax[i].imshow(img1[i][0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i].axis('off')

diff global same local on local

In [ ]:
im = []

t = 1 # No. x maximum
fig = plt.figure(figsize = (5,cls))
ax = fig.subplots(cls,5)
plt.suptitle('local_limited',y = 1)

cols = ['Input(Stimuli)','Conv1','Conv2','Conv3','Conv4']
for axes, col in zip(ax[0], cols):
    axes.set_title(col)
for i in range(9):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  # model.eval()
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])

for i in range(cls):
  for j in range(4):
      ax[i,j+1].imshow(im[i*4+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 

In [ ]:
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

Same glob diff local on local

In [ ]:
img_tmp = img.copy()
img = img1

In [ ]:
im = []

t = 1 # No. x maximum
fig = plt.figure(figsize = (5,cls))
ax = fig.subplots(cls,5)
plt.suptitle('local_limited',y = 1)

cols = ['Input(Stimuli)','Conv1','Conv2','Conv3','Conv4']
for axes, col in zip(ax[0], cols):
    axes.set_title(col)
for i in range(9):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  # model.eval()
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])

for i in range(cls):
  for j in range(4):
      ax[i,j+1].imshow(im[i*4+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 

In [ ]:

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

Diff glob same  local  on global model

In [ ]:
img = img_tmp

In [ ]:
im = []

t = 1 # No. x maximum
fig = plt.figure(figsize = (5,cls))
ax = fig.subplots(cls,5)
plt.suptitle('Global_limited',y = 1)

cols = ['Input(Stimuli)','Conv1','Conv2','Conv3','Conv4']
for axes, col in zip(ax[0], cols):
    axes.set_title(col)
for i in range(9):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  # model.eval()
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  im.append(act[-t,])

for i in range(cls):
  for j in range(4):
      ax[i,j+1].imshow(im[i*4+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 

In [ ]:

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

In [ ]:
im = []
idx = 810
t = 1 # No. x maximum
k = 20 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('global_limited',y = 1)

for i in range(cls):
  id = i + idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  # model.load_state_dict(torch.load("CNN_model/model_CNN_local_muilti_angle"))
  # model.eval()
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

In [ ]:
## 17size
import torch.nn as nn
import torch
from torch import autograd

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch), 
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, input):
        return self.conv(input)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv0 = nn.Conv2d(3, 16, 7, stride=2, padding = 4)  #224
        self.pool0 = nn.MaxPool2d(3,2)
        self.conv1 = DoubleConv(16, 32)  # 56
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(32, 64) # 28
        self.pool2 = nn.MaxPool2d(2)
        # self.conv3 = DoubleConv(64, 128) # 14
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.pool3 = nn.MaxPool2d(2)
        # self.conv4 = DoubleConv(128, 256, padding = 1) # 7
        self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
        self.pool4 = nn.MaxPool2d(2)
        # self.conv5 = DoubleConv(256, 512)
        self.conv5 = nn.Conv2d(256, 256, 3, padding = 1)
        self.fc1 = nn.Linear(256*3*3, 256)
        self.fc2 = nn.Linear(256, cls)
        self.dropout = nn.Dropout(0.5)
        self.dropout0 = nn.Dropout(0.25)
        # self.dropout1 = nn.Dropout(0.5)
    def forward(self, x):
        c0 = self.conv0(x)     
        p0 = F.relu(self.pool0(c0))
        # print(p0.shape)
        c1 = self.conv1(p0)
        p1 = self.pool1(c1)
        # p1 = self.dropout0(p1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        p2 = self.dropout(p2)
        c3 = self.conv3(p2)
        p3 = F.relu(self.pool3(c3))
        p3 = self.dropout(p3)
        c4 = F.relu(self.pool3(self.conv4(p3)))
        c5 = F.relu(self.conv5(c4))
        out = c5.view(-1, 256*3*3)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        # out = nn.LogSoftmax(dim = 1)(x)
        return out

model = CNN()
model.cuda()
model.train()

from torchsummary import summary
summary(model, input_size=(3,224,224))
criterion = nn.CrossEntropyLoss()


In [ ]:
img = []
img1 = []
idx = 810
fig = plt.figure(figsize = (5*cls,5))
ax = fig.subplots(1,cls)
for i in range(cls):
  id = i*9 + idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  img.append(data)
  if i == 2:
    img1.append(data)
    for j in range(1,cls):
      id = i*9 + idx + j
      data, l, g = dataset[id]
      data.unsqueeze_(0)
      data = data.cuda()
      img1.append(data)
  
  ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i].axis('off')

In [ ]:
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv1.register_forward_hook(get_activation('conv1'))
  output = model(data)
  act = activation['conv1'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv2.register_forward_hook(get_activation('conv2'))
  output = model(data)
  act = activation['conv2'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv3.register_forward_hook(get_activation('conv3'))
  output = model(data)
  act = activation['conv3'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()
im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv4.register_forward_hook(get_activation('conv4'))
  output = model(data)
  act = activation['conv4'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv5.register_forward_hook(get_activation('conv5'))
  output = model(data)
  act = activation['conv5'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

im = []
t = 1 # No. x maximum
k = 32 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv5.register_forward_hook(get_activation('conv5'))
  output = model(data)
  act = activation['conv5'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

In [ ]:
act.shape

In [ ]:
im = []
t = 1 # No. x maximum
k = 64 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)
for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv5.register_forward_hook(get_activation('conv5'))
  output = model(data)
  act = activation['conv5'].squeeze().cpu()
  # act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Unsorted')
  for j in range(k):  
      im.append(act[-t-j,])


for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()


im0 = []
t = 1 # No. x maximum
k = 64 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv5.register_forward_hook(get_activation('conv5'))
  output = model(data)
  act = activation['conv5'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  im = []
  for j in range(k):  
      im.append(act[-t-j,])
  im0.append(np.concatenate(im,axis = 1))



for i in range(cls):
  for j in range(k):
      ax[i,j+1].imshow(im[i*k+j])
      ax[i,j+1].axis('off')
plt.tight_layout() 
plt.show()
plt.close()

In [ ]:

im0 = []
t = 1 # No. x maximum
k = 64 # k features shown
fig = plt.figure(figsize = (42,8))
ax = fig.subplots(cls,k + 1)
#plt.suptitle('local_limited',y = 1)

for i in range(cls):
  data = img[i]
  ax[i,0].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i,0].axis('off')
  model.load_state_dict(torch.load("CNN_model/model_CNN_new_global_9angles"))
  model.eval()
  # print('Row %d local orientation'%i,":",l)
  # print('       predicted local orientation',":",model(data).data.max(1)[1].cpu().numpy()[0])
  model.conv5.register_forward_hook(get_activation('conv5'))
  output = model(data)
  act = activation['conv5'].squeeze().cpu()
  act = act.numpy()[np.argsort(np.average(act.numpy().reshape(act.shape[0],-1),axis = 1)),]
  if i == 0 : print('Sorted')
  im = []
  for j in range(k):  
      im.append(act[-t-j,])
  im0.append(np.concatenate(im,axis = 1))
all = np.concatenate(im0,axis = 0)
print(all.shape)

In [ ]:
fig = plt.figure(figsize = (42,8))
plt.imshow(all)